## 描述器

当理解描述器是一个代理的时候，就觉得简单了！

In [1]:
class Int:
    def __init__(self,name):
        self.name = name
    
    def __get__(self, instance, cls):
        if instance is not None:
            return instance.__dict__[self.name]
        return self
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [2]:
class A:
    val = Int('val')
    
    def __init__(self):
        self.val = 3

In [3]:
a = A()

In [4]:
a.val

3

In [5]:
a.__dict__         ## 先从__dict__里找

{'val': 3}

In [6]:
A.__dict__

mappingproxy({'val': <__main__.Int object at 0x7f22b06e4588>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__dict__': <attribute '__dict__' of 'A' objects>, '__init__': <function A.__init__ at 0x7f22b06e0378>, '__module__': '__main__', '__doc__': None})

In [7]:
class Int:
    def __init__(self,name):
        self.name = name
    
    def __get__(self, instance, cls):
        print('get {}'.format(self.name))       ##  新加这一行
        if instance is not None:
            return instance.__dict__[self.name]
        return self
        
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [8]:
class A:
    val = Int('val')
    
    def __init__(self):
        self.val = 3

In [9]:
a = A()

In [10]:
a.val          ##  有点动摇之前学过的东西了，明明__dict__里面有

get val


3

In [11]:
a.__dict__

{'val': 3}

In [ ]:
## 再修改

In [17]:
class Int:
    def __init__(self,name):
        self.name = name
        self.data = {}              
    
    def __get__(self, instance, cls):
        print('get {}'.format(self.name))      
        if instance is not None:
            return self.data[instance]     
        return self
        
    def __set__(self, instance, value):
        self.data[instance] = value
        
    def __str__(self):
        return 'Int'
    
    def __repr__(self):
        return 'Int'

In [18]:
class A:
    val = Int('val')
    
    def __init__(self):
        self.val = 3

In [19]:
a = A()

In [20]:
a.val        ##  当一个类变更 实现了__get__ 和__set__ 方法之后，访问这个类变量会调用__get__方法
             ##  对这个类变量赋值，会调用__set__方法
             ##  这种类变量，就叫做描述器！！！！！！！！！！

get val


3

In [21]:
a.__dict__          ## __dict__ 为空的

{}

In [22]:
Int('val')

Int

In [23]:
a.__dict__['val'] = 5

In [24]:
a.__dict__

{'val': 5}

In [25]:
a.val

get val


3

描述器会提升优先级到`__dict__`之前

暂且认为以上这句话是对的，看下面

In [26]:
## 修改一下类  暂时只有 __get__ 了
class Int:
    def __get__(self, instance, cls):
        return 3

In [31]:
class A:
    val = Int()
    xxx = Int()
    
    def __init__(self):
        self.val = 5

In [32]:
A().__dict__

{'val': 5}

In [33]:
A().val

5

In [34]:
A().xxx

3

描述器会提升优先级到`__dict__`之前，看来这句还是有错误的！

应该改成:带`__set__`方法的描述器会提升优先级到`__dict__`之前！！

In [38]:
class Int:
    def __get__(self, instance, cls):
        return 3
    
    def __set__(self, instance, value):         ## 把__set__也加上
        pass

In [39]:
class A:
    val = Int()
    xxx = Int()
    
    def __init__(self):
        self.val = 5

In [40]:
A().val

3

In [42]:
A().__dict__       ## __dict__里没有

{}

In [43]:
class A:
    val = Int()
    xxx = Int()
    
    def __init__(self):
        self.__dict__['val'] = 5

In [44]:
A().__dict__

{'val': 5}

In [45]:
A().val              ## 还是返回3

3

In [46]:
A().xxx

3

### 描述器总结

描述器：事实上是一种代理机制(像上面val这个变量被Int这个类来代理了)

描述器是一个类，实现了`__get__`、`__set__`或`__delete__`中1个或多个方法，就是描述器！

当一个**类变量**(实际是类成员，注：对实例变量是没用的)被定义为描述器，对这个类变量的操作，将由此描述器来代理

* 访问：`__get__`
* 赋值：`__set__`
* 删除：`__delete__`
* 增加：没有这一说法

当一个描述器实现了`__set__`或`__delete__`方法，优先级会被提升到`__dict__`之前

In [47]:
class A:
    x = Int()        ## Int()是描述器，它实现了__get__  __set__ 等等
    
    def __getattribute__(self, name):
        return 6

In [48]:
A().x

6

经验证：还是`__getattribute__`优先级最高！

这些方法是有固定的签名的！

* `__get__(self, instance, cls)`    instance 表示当前实例  cls 表示类本身，使用类访问的时候，instance为None
* `__set__(self, instance, value)`  instance 表示当前实例，value表示右值，只有实例才会调用`__set__`
* `__delete__(self, instance)`  instance 表示当前实例

只有`__get__`即可以针对类，也可以针对实例；其它两个都只能针对实例！

In [ ]:
看看都代表什么

In [50]:
class Desc:
    def __get__(self, instance, cls):
        print(instance)      ##  打印了实例
        print(cls)           ##  打印了类
        return 'x'

In [51]:
class A:
    x = Desc()

In [52]:
A().x

<class '__main__.A'>


'x'

In [53]:
A.x   ## None 为没有实例呗

None
<class '__main__.A'>


'x'

In [ ]:
再看看__set__的参数

In [61]:
class Desc:
    def __get__(self, instance, cls):
        print(instance)      ##  打印了实例
        print(cls)           ##  打印了类
        return 'x'
    
    def __set__(self, instance, value):
        print(instance)               ##  实例
        print(value)                 ## 赋值

In [62]:
class A:
    x = Desc()

In [63]:
A().x = 5

5


In [64]:
A.x = 5

In [66]:
A().x

5

In [67]:
A.x

5

In [ ]:
再看__delete__ 的参数

In [68]:
class Desc:
    def __get__(self, instance, cls):
        print(instance)      ##  打印了实例
        print(cls)           ##  打印了类
        return 'x'
    
    def __set__(self, instance, value):
        print(instance)               ##  实例
        print(value)  
        
    def __delete__(self, instance):
        print(instance)               ##  实例

In [69]:
class A:
    x = Desc()

In [70]:
del A().x

### 描述器的应用

* classmethod 的实现

In [71]:
from functools import partial       ##  partial 把函数的某些参数固定下来！ 

class Classmethod:
    def __init__(self, fn):
        self.fn = fn
        
    def __get__(self, instance, cls):
        return partial(self.fn, cls)       ## 把它的第一个参数固定为 cls 

In [72]:
class A:
    @Classmethod
    def cls_method(cls):
        print(cls)

In [73]:
A.cls_method()

<class '__main__.A'>


In [74]:
A().cls_method()

<class '__main__.A'>


In [75]:
A.cls_method  ## 相当于A.cls_method.__get__(None, A)

functools.partial(<function A.cls_method at 0x7f22b06e0730>, <class '__main__.A'>)

参照 classmethod 来实现 staticmethod

In [76]:
class Staticmethod:
    def __init__(self, fn):
        self.fn = fn
    
    def __get__(self, instance, cls):
        return self.fn

In [77]:
class A:
    @Staticmethod
    def static_method():
        print('static method')

In [78]:
A.static_method()

static method


In [79]:
A().static_method()      ## OK,实例的跟类的一样

static method


In [80]:
help(A)

Help on class A in module __main__:

class A(builtins.object)
 |  Methods defined here:
 |  
 |  static_method()
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [98]:
class Property:                                                 ## 115分钟做解释
    def __init__(self, fget, fset=None, fdel=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        
    def __get__(self, instance, cls):
        if instance is not None:
            return self.fget(instance)
        return self
    
    def __set__(self, instance, value):
        if callable(self.fset):               ## fset是接受函数
            self.fset(instance, value)
        else:
            raise AttributeError('{} can not assigneable'.format(self.fget.__name__))
            
    def __delete__(self, instance):
        if callable(self.fdel):
            self.fdel(instance)
        else:
            raise AttributeError('{} can not deleteable'.format(self.fget.__name__))
            
    def setter(self, fn):
        self.fset = fn
        return self
    
    def deletter(self, fn):
        self.fdel = fn
        return self

In [99]:
class A:
    def __init__(self):
        self.__x = 1
    
    @Property
    def x(self):
        return self.__x

In [100]:
A().x

1

In [101]:
A().x = 3

AttributeError: x can not assigneable

In [102]:
del A().x      ## 都符合我们的语意

AttributeError: x can not deleteable

In [103]:
class A:
    def __init__(self):
        self.__x = 1
    
    @Property
    def x(self):
        return self.__x
    
    @x.setter
    def x(self, value):
        self.__x = value

In [104]:
A().x

1

In [105]:
a = A()

In [106]:
a.x

1

In [107]:
a.x = 3

In [108]:
a.x

3

In [109]:
class A:
    def __init__(self):
        self.__x = 1
    
    @Property
    def x(self):
        return self.__x
    ## 等价于 x = Property(x)
    
    @x.setter
    def x(self, value):
        self.__x = value
    
    @x.deletter
    def x(self):
        print('can not delete')

In [110]:
a = A()

In [111]:
a.x

1

In [112]:
del a.x     ## 符合语义     

can not delete


所以，property 就是通过描述器来实现的！

* 描述器的使用场景

描述器的使用场景：用于接管对实例变量的操作

数据校验 通过描述器进行类型检查

In [113]:
class Person:
    def __init__(self, name: str, age: int):       ## 类型检查
        self.name = name
        self.age = age

In [114]:
p = Person(18,'comyn')       ##  不做任何检查

In [115]:
import inspect

In [117]:
inspect.signature(Person)        ## 得到签名

<Signature (name:str, age:int)>

In [118]:
sig = inspect.signature(Person) 

In [119]:
sig.parameters

mappingproxy(OrderedDict([('name', <Parameter "name:str">), ('age', <Parameter "age:int">)]))

In [120]:
sig.parameters.items()

odict_items([('name', <Parameter "name:str">), ('age', <Parameter "age:int">)])

In [121]:
list(sig.parameters.items())

[('name', <Parameter "name:str">), ('age', <Parameter "age:int">)]

In [122]:
list(sig.parameters.items())[0]

('name', <Parameter "name:str">)

In [123]:
list(sig.parameters.items())[0][1]

<Parameter "name:str">

In [124]:
param = list(sig.parameters.items())[0][1]

In [125]:
param.annotation

str

这段不完整！

先继续！

### 类的创建与销毁

In [126]:
class A:
    def __init__(self):         ## __init__并不会创建新的对象，而只是初始化它
        print('init')

In [127]:
A()

init


In [128]:
class A:
    def __new__(cls, *args, **kwargs):
        print('new')
        return object.__new__(cls)

In [129]:
A()

new


In [130]:
object.__new__(A)

`object.__new__`用于创建对象

In [ ]:
所以，对象就是由__new__方法创建出来的！！

In [131]:
class A:
    def __new__(cls, *args, **kwargs):    ##  对象的生命周期，由__new__开始，由__init__初始化， 由__del__结束
        print('new')
        return object.__new__(cls)
    
    def __init__(self):
        print('init')
        self.x = 3
        
    def __del__(self):
        print('del')

In [132]:
A()

new
init


In [133]:
a = A()

new
init


In [134]:
del a

del


## 元编程

In [135]:
from collections import namedtuple      ##  命名元组

In [136]:
Person = namedtuple('Person',['name','age'])

神奇之处在于用代码创建了一种新的类型，也就是代码具有写代码的能力

这种能力叫元编程

通过元编程，我们可以控制类创建的过程

类创建的过程：
* 成员 
* 继承列表

### 除非你知道你在做什么，否则不要使用元编程。

重点在描述器，元编程只是拓展！